<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/NN_airfoil_S809_combined_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-auth

In [2]:
from google.colab import auth
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/Data_from_Karim/AirfoilS809/

data_set1.csv  data_set2.csv  data_set3.csv  data_set4.csv  neuralNetwork_reg.py  ReadMe.docx


In [5]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Create a Model class
class Model(nn.Module):
  def __init__(self,in_features=5,h1 = 61, h2 = 61, out_features=1):
    super().__init__()
    self.fc1 = nn.Linear(in_features,h1)
    self.fc2 = nn.Linear(h1,h2)
    self.out = nn.Linear(h2,out_features)

  def forward(self,x):
    x = F.sigmoid(self.fc1(x))
    x = F.sigmoid(self.fc2(x))
    x = self.out(x)

    return x

In [7]:
# torch.manual_seed(42)

model = Model()

In [8]:
# Read the data set - data_set3.csv --> Airfoil S809, Re = 2e6, alpha = 14.2 deg
df1 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set2.csv")
df3 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set3.csv")
df4 = pd.read_csv("/content/drive/MyDrive/Data_from_Karim/AirfoilS809/data_set4.csv")

In [9]:
# Combine data
frames = [df1, df2, df3, df4]
df = pd.concat(frames)


for

,x,y,beta,feature1,feature2,feature3,feature4,feature5
0,0.997919,0.000257,1.0,0.007810,1.780000e-08,0.916840,0.000042,2.360000e-06
1,0.997809,0.000272,1.0,0.013344,1.250000e-08,1.001023,0.000022,4.360000e-06
2,0.997977,0.000226,1.0,0.037124,2.580000e-08,1.001187,0.000011,6.320000e-05
3,0.997921,0.000260,1.0,0.075178,4.870000e-07,0.797050,0.000368,1.820000e-05
4,0.997695,0.000287,1.0,0.005067,1.220000e-08,0.998736,0.000072,4.450000e-07
...,...,...,...,...,...,...,...,...
104173,-18.143043,-2.440683,1.0,6372.480708,9.323091e-01,18.349497,0.001474,4.590000e-10
104174,-16.994571,-1.759553,1.0,372.392258,8.835703e-01,349.630693,0.000511,5.980000e-09
104175,-18.232937,-1.591567,1.0,16925.244250,9.322655e-01,5.337762,0.001316,3.050000e-10
104176,-18.177182,-2.158839,1.0,14216.168940,9.322961e-01,7.214336,0.001455,3.680000e-10
